In [89]:
import time, os, pickle
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.support.select import Select
import chromedriver_autoinstaller
import csv
import random
from selenium.webdriver.common.by import By
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

### chrome

In [90]:
%%time
chromedriver_autoinstaller.install()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 啟動隱藏模式
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.tdcc.com.tw/portal/zh/smWeb/qryStock")
html = driver.page_source
soup = bs(html)

Wall time: 7.42 s


###  查詢函數 (回傳表格)

In [91]:
def tw_stock_share_distribution(query_date, stock_index, select_date_index):
    '''
    query_date: 日期 (用於匯出的df上)
    stock_index: 股票號碼
    select_date_index: 第幾個日期 (用於查詢的滾輪上)
    '''
    try:
        # 設定開啟網頁之日期
        select_1 = Select(driver.find_element(By.NAME,"scaDate"))
        select_1.select_by_index(select_date_index)  # 選定輸入的日期index
        time.sleep(1)

        # 輸入股票代碼
        driver.find_element(By.ID,("StockNo")).clear()
        driver.find_element(By.ID,("StockNo")).send_keys(stock_index)
        time.sleep(1.5)

        # 送出查詢
        driver.find_element(By.XPATH,("//input[@type='submit' and @value='查詢']")).click()
        time.sleep(1.5)

        # 取得網頁原始碼
        html_file = driver.page_source

        # 傳入html file
        bsobj_1 = bs(html_file, "lxml")

        # 找出回傳之分散表
        tbody_trs = bsobj_1.find("table", class_="table").find("tbody").find_all("tr")

        # 每個股票的股權分散表的處理
        stock_table = []

        for tr in tbody_trs:
            tds = tr.find_all("td")
            stock_table.append([td.text for td in tds])   

        df_stock = pd.DataFrame(stock_table)
        df_stock.columns = ["持股分級", "持股數量分級", "人數", "股數", "占集保庫存數比例%"]
        df_stock.insert(0, "資料日期", query_date)
        df_stock.insert(1, "證券代號", stock)
        
        return df_stock
    
    
    except Exception as e:
        print('【系統訊息】{}'.format(e))
        print(f'{stock_index} 檔案  日期 {query_date} 出問題')
        time.sleep(1)
        return None
        pass
        #tw_stock_share_distribution(date_list[i], stock, i)
    

### 爬取的日期

In [92]:
select = soup.find('select',{'id':'scaDate'})

date_list = []
for option in select:
    if option != '\n':
        date_list.append(str(option)[15:23])

        
print(f'從 {date_list[-1]} 到 {date_list[0]}')
print(f'共 {len(date_list)} 天')


從 20210709 到 20220701
共 51 天


### 爬的股票

In [93]:
path_from = os.getcwd() 
stock_list = pd.read_csv(path_from + r'\台股全部4.csv', header=None)[0].tolist()

<hr>

<lr>

## 開始爬

### 1. 先開好網頁

In [94]:
url = r"https://www.tdcc.com.tw/portal/zh/smWeb/qryStock"
driver = webdriver.Chrome()        
driver.get(url)   
time.sleep(2)

### 2. 設定開始爬的股票號碼

In [95]:
start = 5006  # 自己輸入

start_index = int([i for (i, j) in enumerate(stock_list) if j == start][0])
print(f'從第 {start_index+1} 個開始')

從第 265 個開始


### 3. gogo

In [96]:
# 打開之前儲存的錯誤 dict 繼續沿用
if not os.path.exists(r'1_problem_dict.pkl') :
    problem_dict = {}
else:
    with open(r'1_problem_dict.pkl', mode='rb') as inpf :
        try:
            problem_dict = pickle.load(inpf)
        except EOFError:
                problem_dict = {}

In [ ]:
time_start = time.time() #開始計時

for stock in stock_list[start_index:]:
    
    # 建立多層目錄
    path_go = f'{os.getcwd()}\\集保戶股權分散表\\{str(stock)}'
    if not os.path.exists(path_go) :
        os.makedirs(path_go)
            
    for i in range(len(date_list)):       
    
        filename = str(stock) + "-" + str(date_list[i])[0:10] + str(".csv")
        df = tw_stock_share_distribution(date_list[i], stock, i)
        if isinstance(df, type(None)):
            problem_dict[stock] = date_list[i]
            break
        else:
            df.to_csv(f'{path_go}\\{filename}', encoding='cp950', index = False)
            print(f'成功匯出 {filename}')
            
    
    print(f'爬完 {stock} ~~~')
    
    time_end = time.time() 
    timespend= (time_end- time_start)/60
    print('spend '+ str(round(timespend, 2))+ ' min')
    print(' ')

成功匯出 5006-20220701.csv
成功匯出 5006-20220624.csv
成功匯出 5006-20220617.csv
成功匯出 5006-20220610.csv
成功匯出 5006-20220602.csv
成功匯出 5006-20220527.csv
成功匯出 5006-20220520.csv
成功匯出 5006-20220513.csv
成功匯出 5006-20220506.csv
成功匯出 5006-20220429.csv
成功匯出 5006-20220422.csv
成功匯出 5006-20220415.csv
成功匯出 5006-20220408.csv
成功匯出 5006-20220401.csv
成功匯出 5006-20220325.csv
成功匯出 5006-20220318.csv
成功匯出 5006-20220311.csv
成功匯出 5006-20220304.csv
成功匯出 5006-20220225.csv
成功匯出 5006-20220218.csv
成功匯出 5006-20220211.csv
成功匯出 5006-20220128.csv
成功匯出 5006-20220122.csv
成功匯出 5006-20220114.csv
成功匯出 5006-20220107.csv
成功匯出 5006-20211230.csv
成功匯出 5006-20211224.csv
成功匯出 5006-20211217.csv
成功匯出 5006-20211210.csv
成功匯出 5006-20211203.csv
成功匯出 5006-20211126.csv
成功匯出 5006-20211119.csv
成功匯出 5006-20211112.csv
成功匯出 5006-20211105.csv
成功匯出 5006-20211029.csv
成功匯出 5006-20211022.csv
成功匯出 5006-20211015.csv
成功匯出 5006-20211008.csv
成功匯出 5006-20211001.csv
成功匯出 5006-20210924.csv
成功匯出 5006-20210917.csv
成功匯出 5006-20210911.csv
成功匯出 5006-20210903.csv
成功匯出 5006-2

成功匯出 5015-20210827.csv
成功匯出 5015-20210820.csv
成功匯出 5015-20210813.csv
成功匯出 5015-20210806.csv
成功匯出 5015-20210730.csv
成功匯出 5015-20210723.csv
成功匯出 5015-20210716.csv
成功匯出 5015-20210709.csv
爬完 5015 ~~~
spend 30.89 min
 
成功匯出 5016-20220701.csv
成功匯出 5016-20220624.csv
成功匯出 5016-20220617.csv
成功匯出 5016-20220610.csv
成功匯出 5016-20220602.csv
成功匯出 5016-20220527.csv
成功匯出 5016-20220520.csv
成功匯出 5016-20220513.csv
成功匯出 5016-20220506.csv
成功匯出 5016-20220429.csv
成功匯出 5016-20220422.csv
成功匯出 5016-20220415.csv
成功匯出 5016-20220408.csv
成功匯出 5016-20220401.csv
成功匯出 5016-20220325.csv
成功匯出 5016-20220318.csv
成功匯出 5016-20220311.csv
成功匯出 5016-20220304.csv
成功匯出 5016-20220225.csv
成功匯出 5016-20220218.csv
成功匯出 5016-20220211.csv
成功匯出 5016-20220128.csv
成功匯出 5016-20220122.csv
成功匯出 5016-20220114.csv
成功匯出 5016-20220107.csv
成功匯出 5016-20211230.csv
成功匯出 5016-20211224.csv
成功匯出 5016-20211217.csv
成功匯出 5016-20211210.csv
成功匯出 5016-20211203.csv
成功匯出 5016-20211126.csv
成功匯出 5016-20211119.csv
成功匯出 5016-20211112.csv
成功匯出 5016-20211105.csv
成功匯出

成功匯出 5204-20211029.csv
成功匯出 5204-20211022.csv
成功匯出 5204-20211015.csv
成功匯出 5204-20211008.csv
成功匯出 5204-20211001.csv
成功匯出 5204-20210924.csv
成功匯出 5204-20210917.csv
成功匯出 5204-20210911.csv
成功匯出 5204-20210903.csv
成功匯出 5204-20210827.csv
成功匯出 5204-20210820.csv
成功匯出 5204-20210813.csv
成功匯出 5204-20210806.csv
成功匯出 5204-20210730.csv
成功匯出 5204-20210723.csv
成功匯出 5204-20210716.csv
成功匯出 5204-20210709.csv
爬完 5204 ~~~
spend 61.86 min
 
成功匯出 5205-20220701.csv
成功匯出 5205-20220624.csv
成功匯出 5205-20220617.csv
成功匯出 5205-20220610.csv
成功匯出 5205-20220602.csv
成功匯出 5205-20220527.csv
成功匯出 5205-20220520.csv
成功匯出 5205-20220513.csv
成功匯出 5205-20220506.csv
成功匯出 5205-20220429.csv
成功匯出 5205-20220422.csv
成功匯出 5205-20220415.csv
成功匯出 5205-20220408.csv
成功匯出 5205-20220401.csv
成功匯出 5205-20220325.csv
成功匯出 5205-20220318.csv
成功匯出 5205-20220311.csv
成功匯出 5205-20220304.csv
成功匯出 5205-20220225.csv
成功匯出 5205-20220218.csv
成功匯出 5205-20220211.csv
成功匯出 5205-20220128.csv
成功匯出 5205-20220122.csv
成功匯出 5205-20220114.csv
成功匯出 5205-20220107.csv
成功匯出

成功匯出 5212-20211230.csv
成功匯出 5212-20211224.csv
成功匯出 5212-20211217.csv
成功匯出 5212-20211210.csv
成功匯出 5212-20211203.csv
成功匯出 5212-20211126.csv
成功匯出 5212-20211119.csv
成功匯出 5212-20211112.csv
成功匯出 5212-20211105.csv
成功匯出 5212-20211029.csv
成功匯出 5212-20211022.csv
成功匯出 5212-20211015.csv
成功匯出 5212-20211008.csv
成功匯出 5212-20211001.csv
成功匯出 5212-20210924.csv
成功匯出 5212-20210917.csv
成功匯出 5212-20210911.csv
成功匯出 5212-20210903.csv
成功匯出 5212-20210827.csv
成功匯出 5212-20210820.csv
成功匯出 5212-20210813.csv
成功匯出 5212-20210806.csv
成功匯出 5212-20210730.csv
成功匯出 5212-20210723.csv
成功匯出 5212-20210716.csv
成功匯出 5212-20210709.csv
爬完 5212 ~~~
spend 92.66 min
 
成功匯出 5213-20220701.csv
成功匯出 5213-20220624.csv
成功匯出 5213-20220617.csv
成功匯出 5213-20220610.csv
成功匯出 5213-20220602.csv
成功匯出 5213-20220527.csv
成功匯出 5213-20220520.csv
成功匯出 5213-20220513.csv
成功匯出 5213-20220506.csv
成功匯出 5213-20220429.csv
成功匯出 5213-20220422.csv
成功匯出 5213-20220415.csv
成功匯出 5213-20220408.csv
成功匯出 5213-20220401.csv
成功匯出 5213-20220325.csv
成功匯出 5213-20220318.csv
成功匯出

### 印出查無此資料的 dict & 匯出

In [88]:
print(problem_dict)

with open(r'1_problem_dict.pkl', mode='wb') as outf :
    pickle.dump(problem_dict, outf)

{4585: '20220107', 4587: '20220513', 4749: '20220114', 4771: '20211022', 4772: '20220325', 4773: '20220610', 4941: '20210911', 4962: '20220422', 4985: '20220415'}


查無此資料:
1. 4441 檔案  日期 20211112 後 : 沒有表格 (Length mismatch: Expected axis has 1 elements, new values have 5 elements)
2. 4570 檔案  日期 20211119 後 : 沒有表格 (Length mismatch: Expected axis has 1 elements, new values have 5 elements)
3. 4584 檔案  日期 20210723 後 : 沒有表格 (Length mismatch: Expected axis has 1 elements, new values have 5 elements)
4. 4585 檔案  日期 20220107 後 : 沒有表格 (Length mismatch: Expected axis has 1 elements, new values have 5 elements)  
{4585: '20220107', 4587: '20220513', 4749: '20220114', 4771: '20211022', 4772: '20220325', 4773: '20220610', 4941: '20210911', 4962: '20220422', 4985: '20220415'}